In [1]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.least_confidence import LeastConfidence
from al_methods.entropy_sampling import EntropySampling
from al_methods.batch_BALD import BatchBALD
from al_methods.core_set import CoreSet
from ssl_methods.semi_fixmatch import fixmatch
from ssl_methods.semi_flexmatch import flexmatch
from ssl_methods.semi_pseudolabel import pseudolabel
import models
from torchvision import transforms
from framework.framework2 import Framework2
import torch.nn as nn
import time
from torch.utils.data import DataLoader
from scipy.spatial.distance import jensenshannon
import al_methods
import os
import seaborn as sns 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
sns.set_theme()

/home/nasserali/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Configuration

In [2]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },

            'svhn':
                {
                 'n_class':10,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([ 
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.4377, 0.4438, 0.4728), 'std': (0.1980, 0.2010, 0.1970)},
                },
            'cifar10':
                {
                 'n_class':10,
                 'channels':3,
                 'size': 32,
                 'transform_tr': transforms.Compose([
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'loader_tr_args':{'batch_size': 256, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                 'normalize':{'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2470, 0.2435, 0.2616)},
                 },


            'cifar100': 
               {
                'n_class':100,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([
                                transforms.RandomCrop(size = 32, padding=4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'loader_tr_args':{'batch_size': 2048, 'num_workers': 4},
                'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                'normalize':{'mean': (0.5071, 0.4867, 0.4408), 'std': (0.2675, 0.2565, 0.2761)},
                }
        }

### Dataset

In [3]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [4]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()

In [5]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=6 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print("begin with : ",NUM_INIT_LB," Number of round",NUM_ROUND," How many to query",NUM_QUERY)


begin with :  600  Number of round 5  How many to query 600


In [6]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file 
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

### Args

In [7]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.framework='framwork1'
        self.optimizer='SGD'
        self.save_model=False
        self.ALstrat='LeastConfidence'
        self.SSLstrat='fixmatch'
        self.n_epoch=10

In [8]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [9]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

## Models

##### Model 1

In [11]:
model_1=models.__dict__["MobileNet"]()

In [11]:
model_1.feature_extractor.conv1=torch.nn.Conv2d(args_pool['mnist']['channels'],16,kernel_size=3,stride=1,padding=1,bias=False)
model_1.discriminator.dis_fc2=torch.nn.Linear(in_features=50,out_features=args_pool['mnist']['n_class'],bias=True)

#### Model 2

In [12]:
# model_2=models.__dict__["ResNet50"](n_class=args_pool['mnist']['n_class'])

In [13]:
# model_2.feature_extractor.conv1=torch.nn.Conv2d(args_pool['mnist']['channels'],16,kernel_size=3,stride=1,padding=1,bias=False)
# model_2.discriminator.dis_fc2=torch.nn.Linear(in_features=50,out_features=args_pool['mnist']['n_class'],bias=True)

#### Train Model 1

In [14]:
framework_model_1= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model_1, handler, args)

In [15]:
framework_model_1.model_2

ResNet(
  (feature_extractor): resnet_fea(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [16]:
framework_model_1.train(alpha=2e-3,n_epoch=5)

Let's use 1 GPUs!
[Batch=000] [Loss=2.33]

==>>[2023-08-30 08:47:44] [Epoch=000/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=2.49]

==>>[2023-08-30 08:47:45] [Epoch=001/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:11] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=1.75]

==>>[2023-08-30 08:47:47] [Epoch=002/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:06] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=1.21]

==>>[2023-08-30 08:47:48] [Epoch=003/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:03] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=0.96]

==>>[2023-08-30 08:47:50] [Epoch=004/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:01] [LR=0.1000] [Best : Test Accuracy=0.15, Error=0.85]
---- save figure the accuracy/loss curve of train/val into ./save/mnist


0.2801

#### Train Model 2

In [17]:
# framework_model_2= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model_2, handler, args)

In [18]:
framework_model_1.train(alpha=2e-3,n_epoch=5,framework_model_2=True,model_2=framework_model_1.model_2)

Let's use 1 GPUs!
[Batch=000] [Loss=2.38]

==>>[2023-08-30 08:47:52] [Epoch=000/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=2.98]

==>>[2023-08-30 08:47:53] [Epoch=001/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:06] [LR=0.1000] [Best : Test Accuracy=0.28, Error=0.72]
[Batch=000] [Loss=2.32]

==>>[2023-08-30 08:47:55] [Epoch=002/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:04] [LR=0.1000] [Best : Test Accuracy=0.28, Error=0.72]
[Batch=000] [Loss=2.24]

==>>[2023-08-30 08:47:56] [Epoch=003/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:03] [LR=0.1000] [Best : Test Accuracy=0.28, Error=0.72]
[Batch=000] [Loss=2.19]

==>>[2023-08-30 08:47:58] [Epoch=004/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:01] [LR=0.1000] [Best : Test Accuracy=0.28, Error=0.72]
---- save figure the accuracy/loss curve of train/val into ./save/mnist


0.2801

#### Unlabeled data 

In [19]:
# Find indices where the boolean matrix is False
false_indices = np.argwhere(idxs_lb == False).squeeze()

U_X_tr=X_tr[false_indices]
U_Y_tr=Y_tr[false_indices]

unlabeled_data=DataLoader(handler(U_X_tr[:NUM_QUERY],U_Y_tr[:NUM_QUERY],transform=transform_te),batch_size=64)


#### Predict Model 1

In [20]:
model_fr_1=framework_model_1.model.eval()
predictions_model_1 = []
with torch.no_grad():
    for x, _,_ in unlabeled_data:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_model_1.device)  # device could be 'cuda' or 'cpu'
        # Forward pass to obtain predictions
        batch_predictions,_= model_fr_1(x)
        
        # Append batch predictions to the list
        predictions_model_1.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_model_1 = np.concatenate(predictions_model_1, axis=0)


In [21]:
np.argmax(predictions_model_1,axis=1)

array([2, 2, 5, 6, 5, 2, 6, 5, 1, 2, 5, 2, 5, 2, 1, 7, 2, 5, 6, 7, 2, 2,
       4, 4, 2, 5, 5, 5, 5, 5, 5, 5, 6, 7, 2, 2, 2, 2, 5, 2, 1, 2, 7, 7,
       7, 5, 5, 2, 5, 5, 5, 2, 7, 5, 7, 5, 5, 5, 5, 6, 2, 5, 2, 2, 2, 5,
       2, 4, 2, 2, 2, 7, 1, 2, 5, 2, 2, 5, 6, 7, 5, 2, 2, 2, 7, 5, 5, 5,
       2, 5, 2, 7, 5, 2, 5, 2, 7, 5, 1, 5, 7, 1, 7, 1, 2, 2, 2, 2, 2, 5,
       5, 1, 2, 2, 5, 2, 2, 2, 5, 5, 2, 7, 1, 5, 2, 2, 6, 2, 5, 5, 2, 7,
       5, 2, 5, 5, 2, 2, 5, 7, 2, 5, 5, 5, 5, 2, 2, 5, 2, 2, 6, 7, 7, 2,
       2, 2, 7, 2, 5, 2, 5, 5, 2, 2, 2, 5, 5, 2, 5, 2, 7, 5, 1, 2, 7, 2,
       2, 5, 2, 5, 5, 5, 0, 7, 2, 2, 5, 2, 2, 2, 2, 7, 2, 7, 7, 5, 5, 2,
       6, 5, 2, 5, 6, 6, 2, 5, 6, 2, 2, 6, 7, 2, 7, 2, 2, 2, 2, 5, 5, 2,
       2, 7, 2, 5, 5, 5, 5, 2, 7, 6, 2, 2, 2, 2, 5, 2, 2, 5, 5, 2, 2, 7,
       5, 5, 5, 5, 6, 2, 5, 6, 2, 2, 5, 2, 6, 2, 7, 5, 2, 2, 2, 7, 5, 5,
       2, 5, 5, 6, 1, 2, 5, 5, 2, 2, 1, 2, 2, 5, 2, 5, 2, 5, 7, 2, 2, 7,
       2, 6, 2, 2, 2, 2, 5, 2, 2, 5, 2, 5, 7, 2, 2,

#### Predict Model 2

In [22]:
model_fr_2=framework_model_1.model_2.eval()
predictions_model_2 = []
with torch.no_grad():
    for x, _,_ in unlabeled_data:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_model_1.device)  # device could be 'cuda' or 'cpu'
        # Forward pass to obtain predictions
        batch_predictions,_= model_fr_2(x)
        
        # Append batch predictions to the list
        predictions_model_2.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_model_2 = np.concatenate(predictions_model_2, axis=0)


In [23]:
np.argmax(predictions_model_2,axis=1)

array([9, 9, 6, 9, 9, 9, 9, 9, 9, 6, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 6, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       5, 9, 9, 9, 5, 9, 9, 5, 5, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 5, 9, 5, 9, 6, 9, 5, 9, 5, 9, 6, 9, 5, 9, 9, 9, 5, 9, 5, 9,
       5, 9, 9, 9, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 9, 9,
       9, 9, 9, 9, 9, 9, 5, 9, 5, 5, 6, 9, 9, 9, 9, 6, 9, 9, 9, 9, 9, 9,
       5, 9, 9, 9, 9, 9, 9, 9, 6, 9, 9, 9, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9,
       5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       6, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 9,
       9, 9, 9, 9, 6, 9, 5, 9, 9, 5, 5, 9, 9, 9, 9, 9, 5, 6, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 5, 5, 9, 9, 9, 9, 5, 9, 9, 6, 9, 9, 9, 9, 9, 9,
       9, 9, 5, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 6, 5, 9, 9, 9, 9, 9, 9, 9,
       5, 9, 9, 9, 9, 6, 9, 9, 9, 6, 6, 9, 9, 9, 9, 9, 5, 5, 9, 9, 9, 5,
       6, 9, 9, 9, 9, 9, 9, 9, 6, 5, 9, 9, 9, 9, 9,

## Metrics

### Pearson correlation coefficient 

In [ ]:
from scipy import stats
res_pearson=stats.pearsonr(np.argmax(predictions_model_1,axis=1),np.argmax(predictions_model_2,axis=1))

In [ ]:
res_pearson

### SpearMan correlation coeffiecient

In [ ]:
a=np.ones((100,))

In [ ]:
a[2]=0

In [ ]:
b=np.ones((100,))

In [ ]:
stats.pearsonr(a,b)

In [ ]:
res_spearman=stats.spearmanr(np.argmax(predictions_model_1,axis=1),np.argmax(predictions_model_2,axis=1))

In [ ]:
res_spearman

### Cohen's Kappa

In [24]:
a=np.argmax(predictions_model_1,axis=1)

In [25]:
a

array([2, 2, 5, 6, 5, 2, 6, 5, 1, 2, 5, 2, 5, 2, 1, 7, 2, 5, 6, 7, 2, 2,
       4, 4, 2, 5, 5, 5, 5, 5, 5, 5, 6, 7, 2, 2, 2, 2, 5, 2, 1, 2, 7, 7,
       7, 5, 5, 2, 5, 5, 5, 2, 7, 5, 7, 5, 5, 5, 5, 6, 2, 5, 2, 2, 2, 5,
       2, 4, 2, 2, 2, 7, 1, 2, 5, 2, 2, 5, 6, 7, 5, 2, 2, 2, 7, 5, 5, 5,
       2, 5, 2, 7, 5, 2, 5, 2, 7, 5, 1, 5, 7, 1, 7, 1, 2, 2, 2, 2, 2, 5,
       5, 1, 2, 2, 5, 2, 2, 2, 5, 5, 2, 7, 1, 5, 2, 2, 6, 2, 5, 5, 2, 7,
       5, 2, 5, 5, 2, 2, 5, 7, 2, 5, 5, 5, 5, 2, 2, 5, 2, 2, 6, 7, 7, 2,
       2, 2, 7, 2, 5, 2, 5, 5, 2, 2, 2, 5, 5, 2, 5, 2, 7, 5, 1, 2, 7, 2,
       2, 5, 2, 5, 5, 5, 0, 7, 2, 2, 5, 2, 2, 2, 2, 7, 2, 7, 7, 5, 5, 2,
       6, 5, 2, 5, 6, 6, 2, 5, 6, 2, 2, 6, 7, 2, 7, 2, 2, 2, 2, 5, 5, 2,
       2, 7, 2, 5, 5, 5, 5, 2, 7, 6, 2, 2, 2, 2, 5, 2, 2, 5, 5, 2, 2, 7,
       5, 5, 5, 5, 6, 2, 5, 6, 2, 2, 5, 2, 6, 2, 7, 5, 2, 2, 2, 7, 5, 5,
       2, 5, 5, 6, 1, 2, 5, 5, 2, 2, 1, 2, 2, 5, 2, 5, 2, 5, 7, 2, 2, 7,
       2, 6, 2, 2, 2, 2, 5, 2, 2, 5, 2, 5, 7, 2, 2,

In [27]:
b=np.argmax(predictions_model_2,axis=1)

In [ ]:
# a.reshape(-1,1)

In [34]:
# from sklearn.metrics import cohen_kappa_score as am_capa

# _cohen_kappa=am_capa(np.ones((100,)),np.ones((100,)))

/home/nasserali/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in double_scalars
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [33]:
# b[:3]=0

In [40]:
# _cohen_kappa

nan

### Framework 2

In [ ]:
print(f' Strategy for active learning{args.ALstrat} and strategy for semi-supervised learning used {args.SSLstrat}')

stratAl_model_1=LeastConfidence(framework_model_1.X_tr, framework_model_1.Y_tr, framework_model_1.X_te, framework_model_1.Y_te, framework_model_1.idxs_lb, framework_model_1.net, framework_model_1.handler, framework_model_1.args,framework_model_1.n_pool,framework_model_1.device)
stratSSL_model_1=pseudolabel(framework_model_1.X_tr, framework_model_1.Y_tr, framework_model_1.X_te, framework_model_1.Y_te, framework_model_1.idxs_lb, framework_model_1.net, framework_model_1.handler, framework_model_1.args,framework_model_1.n_pool,framework_model_1.device,framework_model_1.predict,framework_model_1.g)

Framework2.train(alpha,n_epochs)

test_acc=framework_model_1.predict(framework_model_1.X_te,framework_model_1.Y_te)
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

for rd in range(1, NUM_ROUND+1):
    
    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_model_1.n_pool)[framework_model_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_model_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_model_1.n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = stratAl.query(NUM_QUERY)
        q_idxs = output
        framework_model_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_model_1.update(framework_model_1.idxs_lb)
        if hasattr(stratAl, 'train'):
        
            best_test_acc=stratAl.train(alpha=2e-3, n_epoch=10)
        else: best_test_acc = framework_model_1.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_model_1.n_pool)[framework_model_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_model_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_model_1.n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = stratSSL.query(NUM_QUERY)
        q_idxs = output
        framework_model_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_model_1.update(framework_model_1.idxs_lb)
        best_test_acc = stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc